# **Final_Tokens_Prompts_Create_a_Simple_Chatbot**

#**Step 1: Install All the Required Packages**

In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 826.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.6/218.6 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00


#**Step 2: Logging with the Hugging Face Account**

In a lot of cases, you must be logged in with a Hugging Face account to interact with the Hub: download private repos, upload files, create PRs,…



#**Appraoch 1**

In [3]:
# !huggingface-cli login

#**Approach 2**

In [2]:
from huggingface_hub import notebook_login

notebook_login()


#**Step 2: Import All the Required Libraries**

In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

#**Step 3: Load the Model**

In [4]:
tokenizer=AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",)
                                        # use_auth_token=True)

# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",)
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
model=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                           device_map='auto',
                                           torch_dtype=torch.float16,)
                                        #    use_auth_token=True)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

#**Step 4: Creating a Pipeline**

#Llama 2 is trained on more data (2 trillion tokens) as compared to Llama 1 and has a context window upto 4K tokens (input token limit)

In [6]:
pipeline=transformers.pipeline("text-generation",
                               model=model,
                               tokenizer=tokenizer,
                               torch_dtype=torch.bfloat16,
                               device_map="auto",
                               # Maximum number of tokens to generate, a word is generally 2-3 tokens (minimum: 1)
                               max_new_tokens=512,
                               #Minimum number of tokens to generate. To disable this set it to -1. A word is generally 2-3 tokens (minimum 1)
                               min_new_tokens=-1,
                               #Adjusts the randomness of output
                               temperature=0.75,
                               do_sample=True,
                               top_k=30,
                               num_return_sequences=1,
                               eos_token_id=tokenizer.eos_token_id)

In [8]:
# pipeline=transformers.pipeline("text-generation",
#                                model=model,
#                                tokenizer=tokenizer,
#                                torch_dtype=torch.bfloat16,
#                                device_map="auto",
#                                # Maximum number of tokens to generate, a word is generally 2-3 tokens (minimum: 1)
#                                max_new_tokens=512,
#                                #Minimum number of tokens to generate. To disable this set it to -1. A word is generally 2-3 tokens (minimum 1)
#                                min_new_tokens=-1,
#                                #Adjusts the randomness of output
#                                temperature=0.75,
#                                do_sample=True,
#                                top_k=30,
#                                num_return_sequences=1,
#                                eos_token_id=tokenizer.eos_token_id)

In [7]:
!nvidia-smi

Wed Jan 17 09:11:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0              26W /  70W |  13021MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f6a9c6df400>, model_kwargs={'temperature': 0})

#**Step 5: Define Instruction Token and System Token**

In [9]:
B_INST, E_INST= "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [10]:
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""

In [11]:
instruction = "Who won the T20 Cricket World Cup 2022"

In [12]:
SYSTEM_PROMPT=B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [13]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [14]:
template

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\nWho won the T20 Cricket World Cup 2022[/INST]"

#**Text Translation**

#Example 1

In [15]:
instruction = "Convert the following text from English to French: \n\n{text}"

In [16]:
SYSTEM_PROMPT=B_SYS+DEFAULT_SYSTEM_PROMPT+E_SYS

In [17]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [18]:
print(template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Convert the following text from English to French: 

{text}[/INST]


In [19]:
prompt = PromptTemplate(input_variables=["text"], template=template)

In [20]:
text ="How are you"

In [21]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [23]:
%%time
LLM_Chain.invoke(text)

CPU times: user 6.79 s, sys: 9.85 ms, total: 6.8 s
Wall time: 7.17 s


{'text': '  Bonjour! I\'m here to help you convert the text from English to French. However, I must inform you that the question "How are you" is not grammatically correct in French. The correct question in French would be "Comment vas-tu?" (pronounced "koh-m-vah-tu").\n\nSo, the converted text from English to French would be:\n\nComment vas-tu?\n\nI hope this helps! Let me know if you have any other questions.'}

#Example 2

In [24]:
instruction = "Convert the following text from English to French: \n\n{text}"

In [25]:
CUSTOM_SYSTEM_PROMPT="You are an advanced assistant that excels at translation."

In [26]:
SYSTEM_PROMPT=B_SYS+CUSTOM_SYSTEM_PROMPT+E_SYS

In [27]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST


In [28]:
print(template)

[INST]<<SYS>>
You are an advanced assistant that excels at translation.
<<SYS>>

Convert the following text from English to French: 

{text}[/INST]


In [29]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [30]:
text ="How are you"

In [31]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [32]:

%%time
output = LLM_Chain.invoke(text)

CPU times: user 1.5 s, sys: 4.44 ms, total: 1.51 s
Wall time: 2.72 s


In [35]:
output1 = output['text']
output1

'  Sure! Here is the translation of "How are you" from English to French:\n\nComment vas-tu?'

In [36]:
import textwrap
wrapped_text = textwrap.fill(output1, width=100)
print(wrapped_text +'\n\n')


  Sure! Here is the translation of "How are you" from English to French:  Comment vas-tu?




#**Summarization**

#Example 3 : Using Default System Prompt

In [37]:
instruction = "Summarize the following text for me : {text}"

In [38]:
SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [39]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [40]:
print(template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Summarize the following text for me : {text}[/INST]


In [41]:
text = """

This article was published as a part of the Data Science Blogathon.

Introduction
Computer vision has advanced considerably but is still challenged in matching the precision of human perception. This article belongs to computer vision. Here we will learn from scratch. It can be challenging for beginners to distinguish between different related computer vision tasks.

Humans can easily detect and identify objects present in an image. The human visual system is fast and accurate and can perform complex tasks like identifying multiple objects and detecting obstacles with little conscious thought. With the availability of large amounts of data, faster GPUs, and better algorithms, we can now easily train computers to detect and classify multiple objects within an image with high accuracy.

With this kind of identification and localization, object detection can be used to count objects in a scene and determine and track their precise locations, all while accurately labeling them.

In this guide, you’ll find answers to all of those questions and more. Whether you’re an experienced machine learning engineer considering implementation, a developer wanting to learn more, or a product manager looking to explore what’s possible with computer vision and object detection, this article is for you.

Let’s Start.
Object Detection
Object detection is a computer vision technique for locating instances of objects in images or videos. Humans can easily detect and identify objects present in an image.


"""

In [42]:
prompt = PromptTemplate(input_variables=["text"],
                      template=template)

In [43]:
LLM_Chain = LLMChain(prompt=prompt, llm=llm)

In [44]:
print(template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Summarize the following text for me : {text}[/INST]


In [45]:
%%time
result = LLM_Chain.invoke(text)

CPU times: user 12.9 s, sys: 53.9 ms, total: 12.9 s
Wall time: 13.6 s


In [47]:
output1 = result['text']
wrapped_text = textwrap.fill(output1, width=100)
print(wrapped_text +'\n\n')

  Sure, I'd be happy to help you summarize the text!  The article discusses the concept of object
detection in computer vision, which involves locating and identifying objects within images or
videos. The human visual system is capable of detecting and identifying objects with ease, but
computer vision algorithms still struggle to match this level of precision. Despite the challenges,
advancements in technology, including larger datasets, faster GPUs, and improved algorithms, have
made it possible to train computers to detect and classify multiple objects within an image with
high accuracy.  The article provides an overview of object detection, including its definition, and
explains how it can be used to count objects in a scene, determine their precise locations, and
accurately label them. It also highlights the potential applications of object detection, making it
a useful resource for machine learning engineers, developers, and product managers interested in
exploring the possibilit

#Example 4: Defining Custom System Prompt

In [48]:
instruction = "Summarize the following text for me : \n\n{text}"
Custom_SYSTEM_PROMPT = "You are an expert in summarizing and expressing key ideas succintly"
SYSTEM_PROMPT = B_SYS + Custom_SYSTEM_PROMPT + E_SYS
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [49]:
print(template)

[INST]<<SYS>>
You are an expert in summarizing and expressing key ideas succintly
<<SYS>>

Summarize the following text for me : 

{text}[/INST]


In [50]:
prompt = PromptTemplate(input_variables=["text"],
                        template=template)

In [51]:
LLM_Chain = LLMChain(prompt=prompt, llm=llm)

In [52]:
%%time
output = LLM_Chain.run(text)

CPU times: user 8 s, sys: 944 µs, total: 8 s
Wall time: 9.38 s


In [54]:
output

"  Sure! Here's a summary of the text you provided:\n\nThe article discusses computer vision and object detection, which involves training computers to detect and classify multiple objects within an image with high accuracy. The human visual system is capable of easily detecting and identifying objects in an image, but computers still struggle to match this level of precision. The article provides an overview of object detection, including its definition and applications, and covers the basics of how it works. It is intended for beginners, experienced machine learning engineers, developers, and product managers who want to learn more about this technology."

In [58]:
%%time
result = LLM_Chain.invoke(text)

CPU times: user 10.2 s, sys: 16.7 ms, total: 10.2 s
Wall time: 12.5 s


In [59]:
result

{'text': '  Sure! Here is a summary of the text you provided:\n\nThe article discusses computer vision and object detection, which involves using algorithms to detect and classify objects within images. The human visual system is capable of effortlessly detecting and identifying objects in images, but computers still struggle to match this level of precision. Despite this challenge, advancements in data availability, GPUs, and algorithms have made it possible to train computers to detect and classify multiple objects within images with high accuracy. This technique has various applications, including counting objects in a scene, determining their precise locations, and accurately labeling them. The article aims to provide a comprehensive guide to object detection, covering its definition, its relationship to other computer vision tasks, and its potential uses.'}

In [60]:
output1 = result['text']
wrapped_text = textwrap.fill(output1, width=100)
print(wrapped_text +'\n\n')

  Sure! Here is a summary of the text you provided:  The article discusses computer vision and
object detection, which involves using algorithms to detect and classify objects within images. The
human visual system is capable of effortlessly detecting and identifying objects in images, but
computers still struggle to match this level of precision. Despite this challenge, advancements in
data availability, GPUs, and algorithms have made it possible to train computers to detect and
classify multiple objects within images with high accuracy. This technique has various applications,
including counting objects in a scene, determining their precise locations, and accurately labeling
them. The article aims to provide a comprehensive guide to object detection, covering its
definition, its relationship to other computer vision tasks, and its potential uses.




#To clean the text you can do

In [ ]:
import textwrap
wrapped_text = textwrap.fill(output, width=100)
print(wrapped_text +'\n\n')


  Sure, I'd be happy to help you summarize the text! Here is a concise summary of the main points:
The article discusses computer vision and object detection, highlighting the challenges of training
computers to detect and classify objects within an image with the same level of accuracy as humans.
Despite these challenges, advancements in technology have made it possible to train computers to
detect and classify multiple objects within an image with high accuracy. The article covers the
basics of object detection, including its definition and how it differs from other computer vision
tasks. It also provides an overview of the techniques used in object detection and the various
applications of this technology. Finally, the article concludes by stating that this guide will
provide detailed answers to various questions related to object detection, including how to
implement it, what it can be used for, and more.




#**Book Summarization**

#Example 5

#Using the Default System Prompt

In [61]:
instruction = "Please provide the summary of the following book {book_name}\n"

In [62]:
B_INST, E_INST= "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [63]:
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
"""

In [64]:
SYSTEM_PROMPT=B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [65]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [66]:
print(template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Please provide the summary of the following book {book_name}
[/INST]


In [67]:
prompt = PromptTemplate(input_variables={'book_name'}, template=template)

In [68]:
LLM_Chain = LLMChain(llm=llm, prompt=prompt)

In [69]:
%%time
result = LLM_Chain.invoke("Alchemist")

CPU times: user 20.3 s, sys: 0 ns, total: 20.3 s
Wall time: 29.5 s


In [70]:
result

{'book_name': 'Alchemist',
 'text': "  Of course! I'd be happy to help you with that.\n\nThe Alchemist is a novel by Paulo Coelho that tells the story of a young shepherd named Santiago, who embarks on a journey to fulfill his personal legend and find his treasure. The book is an allegory, using the metaphor of alchemy to explore themes of spirituality, self-discovery, and the power of belief.\n\nSantiago's journey takes him across the Spanish countryside and eventually to Egypt, where he learns valuable life lessons and discovers his personal legend. Along the way, he meets various characters who teach him about the importance of following his dreams and listening to his heart.\n\nThroughout the book, Coelho explores the idea that each person has a personal legend, or a unique destiny, that they are meant to fulfill. He also emphasizes the importance of following one's intuition and listening to the universe, as these can guide a person towards their personal legend.\n\nThe Alchemist 

In [71]:
output1 = result['text']
wrapped_text = textwrap.fill(output1, width=100)
print(wrapped_text +'\n\n')

  Of course! I'd be happy to help you with that.  The Alchemist is a novel by Paulo Coelho that
tells the story of a young shepherd named Santiago, who embarks on a journey to fulfill his personal
legend and find his treasure. The book is an allegory, using the metaphor of alchemy to explore
themes of spirituality, self-discovery, and the power of belief.  Santiago's journey takes him
across the Spanish countryside and eventually to Egypt, where he learns valuable life lessons and
discovers his personal legend. Along the way, he meets various characters who teach him about the
importance of following his dreams and listening to his heart.  Throughout the book, Coelho explores
the idea that each person has a personal legend, or a unique destiny, that they are meant to
fulfill. He also emphasizes the importance of following one's intuition and listening to the
universe, as these can guide a person towards their personal legend.  The Alchemist has been widely
praised for its inspiring mes

#Example 6

#Using the Custom System Prompt

In [ ]:
#instruction = "Please provide the summary as bullet points for this book {book_name}. Each bullet point should appear in a new line"


In [ ]:
instruction = "Please provide the summary of the following book {book_name}"

In [ ]:
B_INST, E_INST= "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [ ]:
CUSTOM_SYSTEM_PROMPT="""\
You are an expert in Summarizing the text and extracting key information from it
"""

In [ ]:
SYSTEM_PROMPT=B_SYS + CUSTOM_SYSTEM_PROMPT + E_SYS

In [ ]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [ ]:
print(template)

[INST]<<SYS>>
You are an expert in Summarizing the text and extracting key information from it

<<SYS>>

Please provide the summary of the following book {book_name}[/INST]


In [ ]:
prompt = PromptTemplate(input_variables={'book_name'}, template=template)

In [ ]:
LLM_Chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
LLM_Chain.run("Alchemist")

'  Sure, I\'d be happy to provide a summary of the book "The Alchemist" by Paulo Coelho.\n\nSummary:\n\n"The Alchemist" is a novel by Paulo Coelho that tells the story of a young shepherd named Santiago, who embarks on a journey to fulfill his personal legend and find his treasure. The book is set in Spain and North Africa, and follows Santiago as he encounters various spiritual guides and learns about the importance of listening to one\'s heart and following one\'s dreams.\n\nKey Information:\n\n* Santiago, the protagonist, is a young shepherd who has recurring dreams of finding treasure at the Pyramids in Egypt. He sets out on a journey to fulfill his personal legend and find his treasure, despite the doubts and fears of his family and friends.\n* Santiago meets various spiritual guides along the way, including a gypsy woman, a crystal merchant, and an alchemist, who teach him about the importance of listening to one\'s heart and following one\'s dreams.\n* Santiago learns about the 

In [ ]:
output=LLM_Chain.run("Alchemist")

In [ ]:
import textwrap
wrapped_text = textwrap.fill(output, width=100)
#print(wrapped_text +'\n\n')
output = wrapped_text +'\n\n'
output.strip().split("*")

['"The Alchemist" is a novel by Paulo Coelho that tells the story of a young shepherd named\nSantiago, who embarks on a journey to fulfill his Personal Legend and find his treasure. The book is\nan allegory, and its themes include spirituality, self-discovery, and the power of belief. Here is a\nsummary of the key points from the book: 1. The Omens: Santiago experiences a series of omens,\nincluding a recurring dream of a crystal, a fortune teller\'s prophecy, and a mysterious encounter\nwith an alchemist. These omens lead him to believe that he is meant to fulfill a great destiny. 2.\nThe Journey Begins: Santiago decides to follow his dreams and sets out on a journey to find his\ntreasure. He travels to a distant land, facing many challenges and encountering various characters\nalong the way. 3. The Language of the World: Santiago learns that the language of the world is the\nlanguage of the heart, and that he must listen to his own heart to understand the secrets of the\nuniverse. He

# **Creating a Simple Chatbot**

## **Import All the Required Libraries**

#Memory

In [72]:
from langchain.memory import ConversationBufferMemory

In [73]:
instruction = "Chat History:\n\n{chat_history} \n User:\n {user_input}"

Custom_SYSTEM_PROMPT = """You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context"""

In [74]:
SYSTEM_PROMPT=B_SYS + Custom_SYSTEM_PROMPT + E_SYS

In [75]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [76]:
print(template)

[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

{chat_history} 
 User:
 {user_input}[/INST]


In [77]:
prompt=PromptTemplate(input_variables=["chat_history", "user_input"], template=template)

In [78]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [79]:
chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

In [80]:
chain.run(user_input="Hi")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

 
 User:
 Hi[/INST]

> Finished chain.


"  Hello! I'm here to help. What can I assist you with today? 😊"

In [81]:
chain.run(user_input="How is the weather today in North America")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hi
AI:   Hello! I'm here to help. What can I assist you with today? 😊 
 User:
 How is the weather today in North America[/INST]

> Finished chain.


"  Of course! I'm here to help. The current weather conditions in North America are:\n\n* Temperature: Varies depending on the location, but generally ranging from 20°F to 70°F (-7°C to 21°C)\n* Weather Conditions: Sunny, cloudy, rainy, or snowy depending on the location and time of year\n* Humidity: Also varies depending on the location, but generally ranging from 30% to 70%\n\nPlease let me know if you have any other questions!"

In [82]:
chain.run(user_input="If today is Friday What number day of the week is that")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hi
AI:   Hello! I'm here to help. What can I assist you with today? 😊
Human: How is the weather today in North America
AI:   Of course! I'm here to help. The current weather conditions in North America are:

* Temperature: Varies depending on the location, but generally ranging from 20°F to 70°F (-7°C to 21°C)
* Weather Conditions: Sunny, cloudy, rainy, or snowy depending on the location and time of year
* Humidity: Also varies depending on the location, but generally ranging from 30% to 70%

Please let me know if you have any other questions! 
 User:
 If today is Friday What number day of the week is that[/INST]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


"  AI: Hello! I'm here to help. Based on the information provided, today is Friday, which means it is the 5th day of the week. 😊"

In [83]:
chain.run(user_input="Which day of the week is tomorrow")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hi
AI:   Hello! I'm here to help. What can I assist you with today? 😊
Human: How is the weather today in North America
AI:   Of course! I'm here to help. The current weather conditions in North America are:

* Temperature: Varies depending on the location, but generally ranging from 20°F to 70°F (-7°C to 21°C)
* Weather Conditions: Sunny, cloudy, rainy, or snowy depending on the location and time of year
* Humidity: Also varies depending on the location, but generally ranging from 30% to 70%

Please let me know if you have any other questions!
Human: If today is Friday What number day of the week is that
AI:   AI: Hello! I'm here to help. Based on the information provided, today is Friday, which means it is the 5th day of the week. 😊 
 User:
 Which day of the wee

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


"  AI: Hello! I'm here to help. Based on the information provided, tomorrow is Saturday, which means it is the 6th day of the week. 😊"

In [84]:
chain.run(user_input="My name is Moin please remember")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hi
AI:   Hello! I'm here to help. What can I assist you with today? 😊
Human: How is the weather today in North America
AI:   Of course! I'm here to help. The current weather conditions in North America are:

* Temperature: Varies depending on the location, but generally ranging from 20°F to 70°F (-7°C to 21°C)
* Weather Conditions: Sunny, cloudy, rainy, or snowy depending on the location and time of year
* Humidity: Also varies depending on the location, but generally ranging from 30% to 70%

Please let me know if you have any other questions!
Human: If today is Friday What number day of the week is that
AI:   AI: Hello! I'm here to help. Based on the information provided, today is Friday, which means it is the 5th day of the week. 😊
Human: Which day of the week 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


"  Of course, I'll always do my best to assist you, Moin! Is there something else you'd like to know or discuss?"

In [85]:
chain.run(user_input="Can you tell me the meaning of my name Moin")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hi
AI:   Hello! I'm here to help. What can I assist you with today? 😊
Human: How is the weather today in North America
AI:   Of course! I'm here to help. The current weather conditions in North America are:

* Temperature: Varies depending on the location, but generally ranging from 20°F to 70°F (-7°C to 21°C)
* Weather Conditions: Sunny, cloudy, rainy, or snowy depending on the location and time of year
* Humidity: Also varies depending on the location, but generally ranging from 30% to 70%

Please let me know if you have any other questions!
Human: If today is Friday What number day of the week is that
AI:   AI: Hello! I'm here to help. Based on the information provided, today is Friday, which means it is the 5th day of the week. 😊
Human: Which day of the week 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


'  Hello Moin! I\'m here to help you with any questions or information you may need. It\'s important to me that I respect your privacy and only answer your questions to the best of my abilities. Could you please clarify what you would like to know about your name? Perhaps the origin, meaning, or cultural significance of "Moin"? I\'m here to help! 😊'

In [86]:
chain.run(user_input="Can you tell me what is the day after 4 days")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hi
AI:   Hello! I'm here to help. What can I assist you with today? 😊
Human: How is the weather today in North America
AI:   Of course! I'm here to help. The current weather conditions in North America are:

* Temperature: Varies depending on the location, but generally ranging from 20°F to 70°F (-7°C to 21°C)
* Weather Conditions: Sunny, cloudy, rainy, or snowy depending on the location and time of year
* Humidity: Also varies depending on the location, but generally ranging from 30% to 70%

Please let me know if you have any other questions!
Human: If today is Friday What number day of the week is that
AI:   AI: Hello! I'm here to help. Based on the information provided, today is Friday, which means it is the 5th day of the week. 😊
Human: Which day of the week 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


"  Of course! I'd be happy to help. The day after 4 days is... (pausing for effect)...Sunday! 😊"

In [87]:
chain.run(user_input="what is todays date and day, number of day in week as well as this year?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hi
AI:   Hello! I'm here to help. What can I assist you with today? 😊
Human: How is the weather today in North America
AI:   Of course! I'm here to help. The current weather conditions in North America are:

* Temperature: Varies depending on the location, but generally ranging from 20°F to 70°F (-7°C to 21°C)
* Weather Conditions: Sunny, cloudy, rainy, or snowy depending on the location and time of year
* Humidity: Also varies depending on the location, but generally ranging from 30% to 70%

Please let me know if you have any other questions!
Human: If today is Friday What number day of the week is that
AI:   AI: Hello! I'm here to help. Based on the information provided, today is Friday, which means it is the 5th day of the week. 😊
Human: Which day of the week 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


"  Of course! I'd be happy to help. Based on the information provided, today's date is February 18th, 2023. It is the 5th day of the week, which is Friday. This year, today's date falls on the 49th day of the year. 😊 Is there anything else I can help you with?"